In [8]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from scipy.signal import savgol_filter
import cv2



Why do we even whant to smooth our data

In [9]:
# Getting insights into the data structure
columns_str = """time X_NOSE Y_NOSE Z_NOSE visibility_NOSE X_LEFT_EYE_INNER Y_LEFT_EYE_INNER Z_LEFT_EYE_INNER visibility_LEFT_EYE_INNER X_LEFT_EYE Y_LEFT_EYE Z_LEFT_EYE visibility_LEFT_EYE X_LEFT_EYE_OUTER Y_LEFT_EYE_OUTER Z_LEFT_EYE_OUTER visibility_LEFT_EYE_OUTER X_RIGHT_EYE_OUTER Y_RIGHT_EYE_OUTER Z_RIGHT_EYE_OUTER visibility_RIGHT_EYE_OUTER X_RIGHT_EYE Y_RIGHT_EYE Z_RIGHT_EYE visibility_RIGHT_EYE X_RIGHT_EYE_OUTER Y_RIGHT_EYE_OUTER Z_RIGHT_EYE_OUTER visibility_RIGHT_EYE_OUTER X_LEFT_EAR Y_LEFT_EAR Z_LEFT_EAR visibility_LEFT_EAR X_RIGHT_EAR Y_RIGHT_EAR Z_RIGHT_EAR visibility_RIGHT_EAR X_MOUTH_LEFT Y_MOUTH_LEFT Z_MOUTH_LEFT visibility_MOUTH_LEFT X_MOUTH_RIGHT Y_MOUTH_RIGHT Z_MOUTH_RIGHT visibility_MOUTH_RIGHT X_LEFT_SHOULDER Y_LEFT_SHOULDER Z_LEFT_SHOULDER visibility_LEFT_SHOULDER X_RIGHT_SHOULDER Y_RIGHT_SHOULDER Z_RIGHT_SHOULDER visibility_RIGHT_SHOULDER X_LEFT_ELBOW Y_LEFT_ELBOW Z_LEFT_ELBOW visibility_LEFT_ELBOW X_RIGHT_ELBOW Y_RIGHT_ELBOW Z_RIGHT_ELBOW visibility_RIGHT_ELBOW X_LEFT_WRIST Y_LEFT_WRIST Z_LEFT_WRIST visibility_LEFT_WRIST X_RIGHT_WRIST Y_RIGHT_WRIST Z_RIGHT_WRIST visibility_RIGHT_WRIST X_LEFT_PINKY Y_LEFT_PINKY Z_LEFT_PINKY visibility_LEFT_PINKY X_RIGHT_PINKY Y_RIGHT_PINKY Z_RIGHT_PINKY visibility_RIGHT_PINKY X_LEFT_INDEX Y_LEFT_INDEX Z_LEFT_INDEX visibility_LEFT_INDEX X_RIGHT_INDEX Y_RIGHT_INDEX Z_RIGHT_INDEX visibility_RIGHT_INDEX X_LEFT_THUMB Y_LEFT_THUMB Z_LEFT_THUMB visibility_LEFT_THUMB X_RIGHT_THUMB Y_RIGHT_THUMB Z_RIGHT_THUMB visibility_RIGHT_THUMB X_LEFT_HIP Y_LEFT_HIP Z_LEFT_HIP visibility_LEFT_HIP X_RIGHT_HIP Y_RIGHT_HIP Z_RIGHT_HIP visibility_RIGHT_HIP X_LEFT_KNEE Y_LEFT_KNEE Z_LEFT_KNEE visibility_LEFT_KNEE X_RIGHT_KNEE Y_RIGHT_KNEE Z_RIGHT_KNEE visibility_RIGHT_KNEE X_LEFT_ANKLE Y_LEFT_ANKLE Z_LEFT_ANKLE visibility_LEFT_ANKLE X_RIGHT_ANKLE Y_RIGHT_ANKLE Z_RIGHT_ANKLE visibility_RIGHT_ANKLE X_LEFT_HEEL Y_LEFT_HEEL Z_LEFT_HEEL visibility_LEFT_HEEL X_RIGHT_HEEL Y_RIGHT_HEEL Z_RIGHT_HEEL visibility_RIGHT_HEEL X_LEFT_FOOT_INDEX Y_LEFT_FOOT_INDEX Z_LEFT_FOOT_INDEX visibility_LEFT_FOOT_INDEX X_RIGHT_FOOT_INDEX Y_RIGHT_FOOT_INDEX Z_RIGHT_FOOT_INDEX visibility_RIGHT_FOOT_INDEX"""
columns_to_load = columns_str.split()
file_path = "Mediapipe_keypoints.csv"  
df = pd.read_csv(file_path, usecols=columns_to_load)
print("Data Loaded Successfully!")
display(df.head())

Data Loaded Successfully!


,time,X_NOSE,Y_NOSE,Z_NOSE,visibility_NOSE,X_LEFT_EYE_INNER,Y_LEFT_EYE_INNER,Z_LEFT_EYE_INNER,visibility_LEFT_EYE_INNER,X_LEFT_EYE,...,Z_RIGHT_HEEL,visibility_RIGHT_HEEL,X_LEFT_FOOT_INDEX,Y_LEFT_FOOT_INDEX,Z_LEFT_FOOT_INDEX,visibility_LEFT_FOOT_INDEX,X_RIGHT_FOOT_INDEX,Y_RIGHT_FOOT_INDEX,Z_RIGHT_FOOT_INDEX,visibility_RIGHT_FOOT_INDEX
0,0.000000,-0.063733,0.502805,-0.395698,0.999921,-0.083962,0.538291,-0.383986,0.999944,-0.083089,...,0.130854,0.192507,0.018961,-0.489134,-0.100693,0.418703,-0.002238,-0.516333,0.090475,0.193644
1,33.333333,-0.043211,0.502240,-0.394696,0.999912,-0.071669,0.535697,-0.384011,0.999936,-0.070713,...,0.119296,0.194849,0.034104,-0.402429,0.040178,0.397703,-0.005346,-0.358187,0.092969,0.195986
2,66.666667,-0.020163,0.503504,-0.386350,0.999909,-0.051884,0.535895,-0.380524,0.999936,-0.050845,...,0.115370,0.193730,0.053036,-0.289042,0.061571,0.381764,-0.005315,-0.230478,0.111993,0.196811
3,100.000000,-0.008440,0.533132,-0.351903,0.999891,-0.043266,0.560953,-0.352411,0.999932,-0.042234,...,0.198556,0.207998,0.045575,-0.337201,0.151154,0.368794,0.006082,-0.334947,0.215761,0.206893
4,133.333333,0.002767,0.559002,-0.262275,0.999872,-0.036010,0.584854,-0.269395,0.999932,-0.035020,...,0.222347,0.214859,0.037744,-0.280906,0.162776,0.364183,0.015523,-0.267834,0.253445,0.212417


In [10]:

# Define keypoints to load using a split string for clarity
columns_str = """time X_NOSE Y_NOSE Z_NOSE visibility_NOSE X_LEFT_EYE_INNER Y_LEFT_EYE_INNER Z_LEFT_EYE_INNER visibility_LEFT_EYE_INNER X_LEFT_EYE Y_LEFT_EYE Z_LEFT_EYE visibility_LEFT_EYE X_LEFT_EYE_OUTER Y_LEFT_EYE_OUTER Z_LEFT_EYE_OUTER visibility_LEFT_EYE_OUTER X_RIGHT_EYE_OUTER Y_RIGHT_EYE_OUTER Z_RIGHT_EYE_OUTER visibility_RIGHT_EYE_OUTER X_RIGHT_EYE Y_RIGHT_EYE Z_RIGHT_EYE visibility_RIGHT_EYE X_RIGHT_EYE_OUTER Y_RIGHT_EYE_OUTER Z_RIGHT_EYE_OUTER visibility_RIGHT_EYE_OUTER X_LEFT_EAR Y_LEFT_EAR Z_LEFT_EAR visibility_LEFT_EAR X_RIGHT_EAR Y_RIGHT_EAR Z_RIGHT_EAR visibility_RIGHT_EAR X_MOUTH_LEFT Y_MOUTH_LEFT Z_MOUTH_LEFT visibility_MOUTH_LEFT X_MOUTH_RIGHT Y_MOUTH_RIGHT Z_MOUTH_RIGHT visibility_MOUTH_RIGHT X_LEFT_SHOULDER Y_LEFT_SHOULDER Z_LEFT_SHOULDER visibility_LEFT_SHOULDER X_RIGHT_SHOULDER Y_RIGHT_SHOULDER Z_RIGHT_SHOULDER visibility_RIGHT_SHOULDER X_LEFT_ELBOW Y_LEFT_ELBOW Z_LEFT_ELBOW visibility_LEFT_ELBOW X_RIGHT_ELBOW Y_RIGHT_ELBOW Z_RIGHT_ELBOW visibility_RIGHT_ELBOW X_LEFT_WRIST Y_LEFT_WRIST Z_LEFT_WRIST visibility_LEFT_WRIST X_RIGHT_WRIST Y_RIGHT_WRIST Z_RIGHT_WRIST visibility_RIGHT_WRIST X_LEFT_PINKY Y_LEFT_PINKY Z_LEFT_PINKY visibility_LEFT_PINKY X_RIGHT_PINKY Y_RIGHT_PINKY Z_RIGHT_PINKY visibility_RIGHT_PINKY X_LEFT_INDEX Y_LEFT_INDEX Z_LEFT_INDEX visibility_LEFT_INDEX X_RIGHT_INDEX Y_RIGHT_INDEX Z_RIGHT_INDEX visibility_RIGHT_INDEX X_LEFT_THUMB Y_LEFT_THUMB Z_LEFT_THUMB visibility_LEFT_THUMB X_RIGHT_THUMB Y_RIGHT_THUMB Z_RIGHT_THUMB visibility_RIGHT_THUMB X_LEFT_HIP Y_LEFT_HIP Z_LEFT_HIP visibility_LEFT_HIP X_RIGHT_HIP Y_RIGHT_HIP Z_RIGHT_HIP visibility_RIGHT_HIP X_LEFT_KNEE Y_LEFT_KNEE Z_LEFT_KNEE visibility_LEFT_KNEE X_RIGHT_KNEE Y_RIGHT_KNEE Z_RIGHT_KNEE visibility_RIGHT_KNEE X_LEFT_ANKLE Y_LEFT_ANKLE Z_LEFT_ANKLE visibility_LEFT_ANKLE X_RIGHT_ANKLE Y_RIGHT_ANKLE Z_RIGHT_ANKLE visibility_RIGHT_ANKLE X_LEFT_HEEL Y_LEFT_HEEL Z_LEFT_HEEL visibility_LEFT_HEEL X_RIGHT_HEEL Y_RIGHT_HEEL Z_RIGHT_HEEL visibility_RIGHT_HEEL X_LEFT_FOOT_INDEX Y_LEFT_FOOT_INDEX Z_LEFT_FOOT_INDEX visibility_LEFT_FOOT_INDEX X_RIGHT_FOOT_INDEX Y_RIGHT_FOOT_INDEX Z_RIGHT_FOOT_INDEX visibility_RIGHT_FOOT_INDEX"""
columns_to_load = columns_str.split()


CONNECTIONS = [
    #upper 
    ("LEFT_WRIST", "LEFT_INDEX"), ("LEFT_WRIST", "LEFT_THUMB"), ("LEFT_WRIST", "LEFT_PINKY"),
    ("LEFT_WRIST", "LEFT_ELBOW"), ("LEFT_ELBOW", "LEFT_SHOULDER"),
    ("RIGHT_WRIST", "RIGHT_INDEX"), ("RIGHT_WRIST", "RIGHT_THUMB"), ("RIGHT_WRIST", "RIGHT_PINKY"),
    ("RIGHT_WRIST", "RIGHT_ELBOW"), ("RIGHT_ELBOW", "RIGHT_SHOULDER"),
    ("LEFT_SHOULDER", "RIGHT_SHOULDER"),
    
    #lower
    ("LEFT_SHOULDER", "LEFT_HIP"), ("RIGHT_SHOULDER", "RIGHT_HIP"),
    ("LEFT_HIP", "RIGHT_HIP"),
    ("LEFT_HIP", "LEFT_KNEE"), ("RIGHT_HIP", "RIGHT_KNEE"),
    ("LEFT_KNEE", "LEFT_ANKLE"), ("RIGHT_KNEE", "RIGHT_ANKLE"),
    ("LEFT_ANKLE", "LEFT_HEEL"), ("RIGHT_ANKLE", "RIGHT_HEEL"),
    ("LEFT_ANKLE", "LEFT_FOOT_INDEX"), ("RIGHT_ANKLE", "RIGHT_FOOT_INDEX"),
    ("LEFT_HEEL", "LEFT_FOOT_INDEX"), ("RIGHT_HEEL", "RIGHT_FOOT_INDEX")
]

# Normalize landmarks: convert to screen space (640x480) centered on shoulders
def normalize_landmarks(frame_data):
    landmarks = {}
    center_x = (frame_data["X_LEFT_SHOULDER"] + frame_data["X_RIGHT_SHOULDER"]) / 2
    center_y = (frame_data["Y_LEFT_SHOULDER"] + frame_data["Y_RIGHT_SHOULDER"]) / 2
    center_z = (frame_data["Z_LEFT_SHOULDER"] + frame_data["Z_RIGHT_SHOULDER"]) / 2

    for col in frame_data.index:
        if col.startswith("X_"):
            key = col.replace("X_", "")
            x = (frame_data[col] - center_x) * 640 + 320
            y = (frame_data[f"Y_{key}"] - center_y) * 480 + 240
            z = frame_data[f"Z_{key}"] - center_z
            landmarks[key] = (int(x), int(y), z)
    return landmarks
'''
the Commented code Is a real time playback of the key points for my better understanding of data
'''
# Real-Time Playback
fps = 30
frame_delay = int(1000 / fps)

for i in range(len(df)):
    frame = np.zeros((1020, 640, 3), dtype=np.uint8)
    frame_data = df.iloc[i]
    landmarks = normalize_landmarks(frame_data)

    # Draw Connections
    for (p1, p2) in CONNECTIONS:
        if p1 in landmarks and p2 in landmarks:
            cv2.line(frame, landmarks[p1][:2], landmarks[p2][:2], (255, 255, 255), 2)

    # Draw Keypoints with depth-aware sizing and brightness
    sorted_landmarks = sorted(landmarks.items(), key=lambda item: item[1][2], reverse=True)  # Sort by z value

    for key, (x, y, z) in sorted_landmarks:
        z_scale = max(1, 10 - int(z * 10))
        brightness = max(50, min(255, int(255 - abs(z) * 200)))

        if "WRIST" in key:
            color = (0, 0, brightness)
        elif "ELBOW" in key:
            color = (brightness, 0, 0)
        elif "EYE" in key:
            color = (0, 0, brightness)
        elif "EAR" in key:
            color = (0, brightness, 0)
        elif "SHOULDER" in key:
            color = (0, brightness, brightness)
        elif "MOUTH" in key:
            color = (0, brightness, brightness)
        elif "NOSE" in key:
            color = (brightness, 0, 0)
        elif any(finger in key for finger in ["THUMB", "INDEX", "PINKY"]):
            color = (0, brightness, 0)
        else:
            color = (brightness, brightness, brightness)

        cv2.circle(frame, (x, y), z_scale, color, -1)

    # Display the corresponding time value in the top left corner
    time_value = frame_data["time"]
    cv2.putText(frame, f'Time: {time_value}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow("3D Depth-Aware Skeleton", frame)
    if cv2.waitKey(frame_delay) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

2025-04-03 15:21:44.078 python[43490:10529072] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-03 15:21:44.078 python[43490:10529072] +[IMKInputSession subclass]: chose IMKInputSession_Modern
